In [1]:
import urllib.request
from PIL import Image
import numpy as np
import cv2
from keras.models import load_model


Using TensorFlow backend.


In [3]:
from selenium import webdriver
from selenium.common.exceptions import NoAlertPresentException
from bs4 import BeautifulSoup
import functools as ft
import re
import csv
import datetime


import urllib.request
from PIL import Image
import numpy as np
import cv2
from keras.models import load_model
from io import BytesIO

model = load_model('nn_captcha.h5')

def give_captcha(fname):
    left=60
    upper=11
    lower=22
    img=Image.open(fname).convert('L')
    X=[]
    for i in range(1,6):
        right=left+8
        croppedimg=img.crop((left,upper,right,lower))
        np_im = np.array(croppedimg)
        resized_img=cv2.resize(np_im,(28,28), interpolation = cv2.INTER_CUBIC)
        X.append(resized_img.tolist())
        left=right+1
    X=np.array(X)
    num_pixels = X.shape[1] * X.shape[2]
    X = X.reshape(X.shape[0], num_pixels).astype('float32')
    res=model.predict_classes(X)
    captcha=''.join(str(e) for e in res)
    return captcha



#converts marks to grade point
def grade(n):
	if n==100:
		return n//10
	if n>=45:
		return n//10+1
	elif n>=40 and n<45:
		return n//10
	else:
		return 0

#list to store USN, name and calculated sgpa of all students
sgpa=[]

#regular expression to find Student name:
nm='<tdstyle="padding-left:15px"><b>:</b>(\w+[.]?\w+)</td>'
regex=re.compile(nm)

#last valid USN
last=198

url='http://results.vtu.ac.in/resultsvitavicbcs_19/index.php'

driver=webdriver.Chrome(executable_path=r'C:\Users\ankur\Desktop\captcha\chromedriver.exe')   # Create a new instance of the Chrome driver 

# session_url = driver.command_executor._url  
# session_id = driver.session_id
# driver.get(url)

# driver.implicitly_wait(10)
t1=datetime.datetime.now()

for i in range(1,last+1):
	
	# driver = webdriver.Remote(command_executor=session_url,desired_capabilities={})
	# driver.session_id = session_id
	# driver.get(url)

	#generating the usn
	usn='1cr15cs'
	if i<10:
		usn=usn+'00'+str(i)
	elif i>=10 and i<100:
		usn=usn+'0'+str(i)
	else:
		usn+=str(i)

	driver.get(url)	#fetches the page
	driver.save_screenshot('my_page.png')

	img = driver.find_element_by_xpath('/html/body/div[2]/div/div[2]/div/div/form/div/div[2]/div/div[3]/img')
	location = img.location
	size = img.size
	png = driver.get_screenshot_as_png()
	im = Image.open(BytesIO(png))

	left = location['x']
	top = location['y']
	right = location['x'] + size['width']
	bottom = location['y'] + size['height']
	im = im.crop((left, top, right, bottom)) # defines crop points
	fname='mah_screenshot.png'
	im.save(fname)

	captcha=give_captcha(fname)
	print(captcha)
	ele=driver.find_element_by_name('lns')
	ele.send_keys(usn)
	ele=driver.find_element_by_name('captchacode')
	ele.send_keys(captcha)
	
	try:
		#try checks if the usn is valid or not
		driver.find_element_by_id("submit").click()				
		doc=driver.page_source									#gets the page source the web-site

		doc1=ft.reduce(lambda x,y:x+y,doc.split())				#getting the page source as a string without spaces
		
		x=re.search(regex,doc1)
		if x:
			name=x.group(1)
		
		# driver.quit()

		if re.search(r'Semester:7',doc1):						#checks if the semester is 5
			soup=BeautifulSoup(doc,'lxml')						#creating the BeautifulSoup object
			b=soup.find('div',class_='divTableBody')
			bc1=b.find('div',class_='divTableRow')
			bc3=bc1.findNextSiblings()
			l=[]
			for i in range(3):
				bc4=bc3[i].findChildren()
				x=bc4[4].text
				l.append(x.rstrip(' '))
			
			l=list(map(int,l))									# l contains all subjects marks of a student
			print(l)
			gpa=0.0

			for i in range(0,len(l)):
				if i<3:
					gpa+=(grade(l[i])*4.0)
				elif i<5:
					gpa+=(grade(l[i])*3.0)
				else:
					gpa+=(grade(l[i])*2.0)

			sgpa.append((usn,name.strip(),str(gpa/24.0)))

	except NoAlertPresentException as e:
		continue
	except Exception as e:
		print(e)
		alert = driver.switch_to.alert
		alert.accept()

	
driver.quit()

t2=datetime.datetime.now()
print(t2-t1)
#sort the list in decreasing order
sgpa=sorted(sgpa,key=lambda x:x[2],reverse=True)
print(sgpa)

#writing the list to a csv file
f=open('cmr-sgpa.csv','w',newline='')
writer=csv.writer(f)
writer.writerow(['USN','Name','SGPA'])
for i in range(len(sgpa)):
	writer.writerow([sgpa[i][0],sgpa[i][1],sgpa[i][2]])
f.close()

In [ ]:
# import datetime

In [ ]:
# x1=datetime.datetime.now()
# for i in range(1000000000):
#     pass
# x2=datetime.datetime.now()
# print(x2-x1)

In [15]:
model = load_model('nn_captcha.h5')

def give_captcha(fname):
    left=60
    upper=11
    lower=22
    img=Image.open(fname).convert('L')
    X=[]
    for i in range(1,6):
        right=left+8
        croppedimg=img.crop((left,upper,right,lower))
        np_im = np.array(croppedimg)
        resized_img=cv2.resize(np_im,(28,28), interpolation = cv2.INTER_CUBIC)
        X.append(resized_img.tolist())
        left=right+1
    X=np.array(X)
    num_pixels = X.shape[1] * X.shape[2]
    X = X.reshape(X.shape[0], num_pixels).astype('float32')
    res=model.predict_classes(X)
    captcha=''.join(str(e) for e in res)
    return captcha


url='http://results.vtu.ac.in/revalresultvitavicbcs19/index.php'
driver=webdriver.Chrome(executable_path=r'C:\Users\ankur\Desktop\captcha\chromedriver.exe')

driver.get(url)	#fetches the page
driver.save_screenshot('my_page.png')

img = driver.find_element_by_xpath('/html/body/div[2]/div/div[2]/div/div/form/div/div[2]/div/div[3]/img')
location = img.location
size = img.size
png = driver.get_screenshot_as_png()
im = Image.open(BytesIO(png))

left = location['x']
top = location['y']
right = location['x'] + size['width']
bottom = location['y'] + size['height']
im = im.crop((left, top, right, bottom)) # defines crop points
fname='mah_screenshot.png'
im.save(fname)

captcha=give_captcha(fname)
print(captcha)
ele=driver.find_element_by_name('lns')
ele.send_keys('1cr15cs002')
ele=driver.find_element_by_name('captchacode')
ele.send_keys(captcha)


driver.find_element_by_id("submit").click()
doc=driver.page_source
soup=BeautifulSoup(doc,'lxml')
b=soup.find('div',class_='divTableBody')
bc1=b.find('div',class_='divTableRow')
bc3=bc1.findNextSiblings()
l=[]
print(bc3)
for i in range(len(bc3)):
    bc4=bc3[i].findChildren()
    x=bc4[2].text
    l.append(x.rstrip(' '))


46815
[<div class="divTableRow">
<div class="divTableCell">15CS71</div>
<div class="divTableCell" style="text-align: left;width: 400px;">WEB TECHNOLOGY AND ITS APPLICATIONS</div>
<div class="divTableCell" style="text-align: center;width: 400px;">37</div>
<div class="divTableCell" style="text-align: center;width: 400px;">28</div>
<div class="divTableCell" style="text-align: center;">15</div>
<div class="divTableCell">P</div>
</div>, <div class="divTableRow">
<div class="divTableCell">15CS72</div>
<div class="divTableCell" style="text-align: left;width: 400px;">ADVANCED COMPUTER ARCHITECTURES</div>
<div class="divTableCell" style="text-align: center;width: 400px;">30</div>
<div class="divTableCell" style="text-align: center;width: 400px;">19</div>
<div class="divTableCell" style="text-align: center;">12</div>
<div class="divTableCell">P</div>
</div>]


In [16]:
print(l)

['37', '30']


In [14]:
xx[0]

<div class="divTableCell">15CS71</div>

In [17]:
from selenium import webdriver
from selenium.common.exceptions import NoAlertPresentException
from bs4 import BeautifulSoup
import functools as ft
import re
import csv
import datetime


import urllib.request
from PIL import Image
import numpy as np
import cv2
from keras.models import load_model
from io import BytesIO

model = load_model('nn_captcha.h5')

def give_captcha(fname):
    left=60
    upper=11
    lower=22
    img=Image.open(fname).convert('L')
    X=[]
    for i in range(1,6):
        right=left+8
        croppedimg=img.crop((left,upper,right,lower))
        np_im = np.array(croppedimg)
        resized_img=cv2.resize(np_im,(28,28), interpolation = cv2.INTER_CUBIC)
        X.append(resized_img.tolist())
        left=right+1
    X=np.array(X)
    num_pixels = X.shape[1] * X.shape[2]
    X = X.reshape(X.shape[0], num_pixels).astype('float32')
    res=model.predict_classes(X)
    captcha=''.join(str(e) for e in res)
    return captcha



#converts marks to grade point
def grade(n):
	if n==100:
		return n//10
	if n>=45:
		return n//10+1
	elif n>=40 and n<45:
		return n//10
	else:
		return 0

#list to store USN, name and calculated sgpa of all students
sgpa=[]

#regular expression to find Student name:
nm='<tdstyle="padding-left:15px"><b>:</b>(\w+[.]?\w+)</td>'
regex=re.compile(nm)

#last valid USN
last=198

url='http://results.vtu.ac.in/revalresultvitavicbcs19/index.php'

driver=webdriver.Chrome(executable_path=r'C:\Users\ankur\Desktop\captcha\chromedriver.exe')   # Create a new instance of the Chrome driver 

# session_url = driver.command_executor._url  
# session_id = driver.session_id
# driver.get(url)

# driver.implicitly_wait(10)
t1=datetime.datetime.now()

for i in range(1,last+1):
	
	# driver = webdriver.Remote(command_executor=session_url,desired_capabilities={})
	# driver.session_id = session_id
	# driver.get(url)

	#generating the usn
	usn='1cr15cs'
	if i<10:
		usn=usn+'00'+str(i)
	elif i>=10 and i<100:
		usn=usn+'0'+str(i)
	else:
		usn+=str(i)

	driver.get(url)	#fetches the page
	driver.save_screenshot('my_page.png')

	img = driver.find_element_by_xpath('/html/body/div[2]/div/div[2]/div/div/form/div/div[2]/div/div[3]/img')
	location = img.location
	size = img.size
	png = driver.get_screenshot_as_png()
	im = Image.open(BytesIO(png))

	left = location['x']
	top = location['y']
	right = location['x'] + size['width']
	bottom = location['y'] + size['height']
	im = im.crop((left, top, right, bottom)) # defines crop points
	fname='mah_screenshot.png'
	im.save(fname)

	captcha=give_captcha(fname)
	print(captcha)
	ele=driver.find_element_by_name('lns')
	ele.send_keys(usn)
	ele=driver.find_element_by_name('captchacode')
	ele.send_keys(captcha)
	
	try:
		#try checks if the usn is valid or not
		driver.find_element_by_id("submit").click()				
		doc=driver.page_source									#gets the page source the web-site

		doc1=ft.reduce(lambda x,y:x+y,doc.split())				#getting the page source as a string without spaces
		
		x=re.search(regex,doc1)
		if x:
			name=x.group(1)
		
		# driver.quit()

		if re.search(r'Semester:7',doc1):						#checks if the semester is 5
			soup=BeautifulSoup(doc,'lxml')						#creating the BeautifulSoup object
			b=soup.find('div',class_='divTableBody')
			bc1=b.find('div',class_='divTableRow')
			bc3=bc1.findNextSiblings()
			l=[]
			for i in range(len(bc3)):
				bc4=bc3[i].findChildren()
				x=bc4[2].text
				l.append(x.rstrip(' '))
			
			l=list(map(int,l))									# l contains all subjects marks of a student
			print(l)
			gpa=0.0

			for i in range(0,len(l)):
				if i<3:
					gpa+=(grade(l[i])*4.0)
				elif i<5:
					gpa+=(grade(l[i])*3.0)
				else:
					gpa+=(grade(l[i])*2.0)

			sgpa.append((usn,name.strip(),str(gpa/24.0)))

	except NoAlertPresentException as e:
		continue
	except Exception as e:
		print(e)
		alert = driver.switch_to.alert
		alert.accept()

	
driver.quit()

t2=datetime.datetime.now()
print(t2-t1)
#sort the list in decreasing order
sgpa=sorted(sgpa,key=lambda x:x[2],reverse=True)
print(sgpa)

#writing the list to a csv file
# f=open('cmr-sgpa-reval.csv','w',newline='')
# writer=csv.writer(f)
# writer.writerow(['USN','Name','SGPA'])
# for i in range(len(sgpa)):
# 	writer.writerow([sgpa[i][0],sgpa[i][1],sgpa[i][2]])
# f.close()

81492
Alert Text: None
Message: unexpected alert open: {Alert text : University Seat Number is not available or Invalid..!}
  (Session info: chrome=73.0.3683.103)
  (Driver info: chromedriver=2.46.628402 (536cd7adbad73a3783fdc2cab92ab2ba7ec361e1),platform=Windows NT 10.0.17134 x86_64)

53788
[37, 30]
11338
Alert Text: None
Message: unexpected alert open: {Alert text : University Seat Number is not available or Invalid..!}
  (Session info: chrome=73.0.3683.103)
  (Driver info: chromedriver=2.46.628402 (536cd7adbad73a3783fdc2cab92ab2ba7ec361e1),platform=Windows NT 10.0.17134 x86_64)

97553
Alert Text: None
Message: unexpected alert open: {Alert text : University Seat Number is not available or Invalid..!}
  (Session info: chrome=73.0.3683.103)
  (Driver info: chromedriver=2.46.628402 (536cd7adbad73a3783fdc2cab92ab2ba7ec361e1),platform=Windows NT 10.0.17134 x86_64)

58899
[41, 23, 18]
36837
[28]
63658
[44, 52]
13425
[30]
24638
Alert Text: None
Message: unexpected alert open: {Alert text :

43633
[40, 28, 28, 52, 50]
87159
Alert Text: None
Message: unexpected alert open: {Alert text : University Seat Number is not available or Invalid..!}
  (Session info: chrome=73.0.3683.103)
  (Driver info: chromedriver=2.46.628402 (536cd7adbad73a3783fdc2cab92ab2ba7ec361e1),platform=Windows NT 10.0.17134 x86_64)

28928
Alert Text: None
Message: unexpected alert open: {Alert text : University Seat Number is not available or Invalid..!}
  (Session info: chrome=73.0.3683.103)
  (Driver info: chromedriver=2.46.628402 (536cd7adbad73a3783fdc2cab92ab2ba7ec361e1),platform=Windows NT 10.0.17134 x86_64)

72778
[46, 37, 48]
88391
Alert Text: None
Message: unexpected alert open: {Alert text : University Seat Number is not available or Invalid..!}
  (Session info: chrome=73.0.3683.103)
  (Driver info: chromedriver=2.46.628402 (536cd7adbad73a3783fdc2cab92ab2ba7ec361e1),platform=Windows NT 10.0.17134 x86_64)

73848
Alert Text: None
Message: unexpected alert open: {Alert text : University Seat Number i

89842
[44, 16, 41, 37]
37733
Alert Text: None
Message: unexpected alert open: {Alert text : University Seat Number is not available or Invalid..!}
  (Session info: chrome=73.0.3683.103)
  (Driver info: chromedriver=2.46.628402 (536cd7adbad73a3783fdc2cab92ab2ba7ec361e1),platform=Windows NT 10.0.17134 x86_64)

67914
Alert Text: None
Message: unexpected alert open: {Alert text : University Seat Number is not available or Invalid..!}
  (Session info: chrome=73.0.3683.103)
  (Driver info: chromedriver=2.46.628402 (536cd7adbad73a3783fdc2cab92ab2ba7ec361e1),platform=Windows NT 10.0.17134 x86_64)

65269
[12, 21, 16]
22648
Alert Text: None
Message: unexpected alert open: {Alert text : University Seat Number is not available or Invalid..!}
  (Session info: chrome=73.0.3683.103)
  (Driver info: chromedriver=2.46.628402 (536cd7adbad73a3783fdc2cab92ab2ba7ec361e1),platform=Windows NT 10.0.17134 x86_64)

31876
[20]
83629
[39, 19, 28]
85833
Alert Text: None
Message: unexpected alert open: {Alert text 

68626
[55]
54245
[55, 44, 30]
66587
[18, 35]
94197
[53, 19, 53]
64335
Alert Text: None
Message: unexpected alert open: {Alert text : University Seat Number is not available or Invalid..!}
  (Session info: chrome=73.0.3683.103)
  (Driver info: chromedriver=2.46.628402 (536cd7adbad73a3783fdc2cab92ab2ba7ec361e1),platform=Windows NT 10.0.17134 x86_64)

86658
[18]
29322
Alert Text: None
Message: unexpected alert open: {Alert text : University Seat Number is not available or Invalid..!}
  (Session info: chrome=73.0.3683.103)
  (Driver info: chromedriver=2.46.628402 (536cd7adbad73a3783fdc2cab92ab2ba7ec361e1),platform=Windows NT 10.0.17134 x86_64)

59229
[18]
51932
Alert Text: None
Message: unexpected alert open: {Alert text : University Seat Number is not available or Invalid..!}
  (Session info: chrome=73.0.3683.103)
  (Driver info: chromedriver=2.46.628402 (536cd7adbad73a3783fdc2cab92ab2ba7ec361e1),platform=Windows NT 10.0.17134 x86_64)

13132
[59, 43]
12828
[28]
39965
Alert Text: None
Mess